Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

Calculando a derivada fixando os parâmetros iguais aos de Dou et al. e substituindo pelo $\beta$ encontrado na última estimação com 2 clusters

In [1]:
using LinearAlgebra, Statistics

using DataFrames
using BenchmarkTools


In [18]:
using Distributed
nprocs = 2
addprocs(nprocs)
pids = workers()
aux = pids[nprocs] #processador auxiliar

3

In [19]:
@everywhere using NBInclude
@everywhere @nbinclude("solve_model.ipynb")

#importou a função solve_tree

In [20]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj
game_parameters = [0.93, 9.03, 0.1, 0.96];

In [21]:
A2 = @spawnat aux solve_tree(data2, game_parameters);
A1 = solve_tree(data1, game_parameters);
A2 = fetch(A2);

212.426361 seconds (634.63 M allocations: 14.094 GiB      From worker 3:	196.170955 seconds (576.80 M allocations: 12.816 GiB, 1.23% gc time)
, 1.25% gc time)


Simulation parameters

In [22]:
#θs0, θj0, μ
simulation_parameters = [50.0, 28.0, 3.04];

### Criando funções para simular o jogo

In [23]:
#cost function
function Ct(t, c0=c0, c1=c1)
    
    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
    
    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
    
    if(propositor=="s")
        hst = hk_next
        lst = lk_next
        
        hjt = hm_next
        ljt = lm_next
        
    else
        hst = hm_next
        lst = lm_next
        
        hjt = hk_next
        ljt = lk_next
    end
    
    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [24]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == grid)
        
        return grid

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [25]:
function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




# #test
# Hs0 = 30
# Hj0 = 28


# hst = Hs0
# hjt = Hj0

# #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
# lst = hst
# ljt = hjt




# choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [26]:
function simulate_game(data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    
    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj
    
    
    c0 = game_parameters[3];
    c1 = c0/30;

    ρ = game_parameters[1];
    λj = game_parameters[4];

    
    
#     hst = Hs0
#     hjt = Hj0
    
    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(5);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
        
        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
            
            

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [27]:
#unit testing
@btime simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])

  3.100 μs (19 allocations: 838 bytes)


5-element Array{Float64,1}:
 0.0489792435426405
 0.15913461311890698
 3.0
 9.0
 6.0

In [28]:
function loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    
    
    
    #S is the number of simulations
    #N is the number of observations
    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster
    
    
    
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas
    
    
    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:5] .= simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end
            
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [29]:
#unit testing
S = 40
N = 66
Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

40×66×7 Array{Float64,3}:
[:, :, 1] =
 0.0485328  0.0466646  0.0380385  0.0857857  …  0.375579  0.382038  0.356152
 0.148574   0.0530265  0.056274   0.144478      0.373842  0.366049  0.377406
 0.0472514  0.0435052  0.0532849  0.054255      0.364637  0.354206  0.378065
 0.0529259  0.0568394  0.156266   0.041024      0.49104   0.365899  0.753183
 0.0473318  0.133397   0.0544861  0.0477753     0.644374  0.386269  0.38766
 0.0546754  0.0509281  0.0526243  0.0628804  …  0.379711  0.358951  0.38105
 0.0539361  0.0485439  0.0496024  0.0441805     0.554909  0.417644  0.361867
 0.0387041  0.0385105  0.0578878  0.0578878     0.30326   0.363147  0.372179
 0.0373517  0.0589351  0.0502097  0.0530265     0.390176  0.341177  0.556852
 0.048209   0.042456   0.0521228  0.0455587     0.38258   0.326061  0.364975
 0.0408574  0.0506966  0.0628804  0.0465906  …  0.344596  0.365511  0.32014
 0.0495681  0.0568055  0.0489942  0.0475396     0.370891  0.377552  0.347979
 0.0493914  0.0549891  0.054702   0.05247

In [30]:
function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    Moments = zeros(S, 8)
    
    μ = simulation_parameters[3]
    for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Ds = Results[s,:,6];
            DF.Dj = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = size(INCOURT_REORG,1)/ size(INCOURT,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)



            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)
        
        
            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]

            
            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

        end
    return vec(mean(Moments, dims=1))
end



  

simulate_moments (generic function with 1 method)

In [42]:
using Random

Random.seed!(2021)

#test
S = 100
N = 66

simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

8-element Array{Float64,1}:
 1.521268561566244
 1.0
 2.7206538627894252
 0.9065151515151514
 0.6353218636993916
 0.3637251645638282
 0.7452142396880221
 0.5273065718374672

Calculando a derivada dos momentos em relação aos parâmetros

In [32]:
#data_moments
data_moments = [1.479, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497];

In [33]:
function simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    println("parameter ",i, game_parameters, simulation_parameters)
        
    θs0, θj0, μ = simulation_parameters;
    θs0 = Int64(round(θs0, digits=0));
    θj0 = Int64(round(θj0, digits=0));

    λj = game_parameters[4];
    new_simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1) 
    
    return new_simulated_moments
end

simulate_new_moments (generic function with 1 method)

In [43]:
# calculando a derivada em relação a ρ

function derivative_moments(S, N, game_parameters, simulation_parameters, simulated_moments, data1, data2, n1, aux)
    
    
    P = size(game_parameters,1)+size(simulation_parameters,1)
    M = size(simulated_moments,1)
    D = zeros(M, P)
    
    estimated_game_parameters = game_parameters
    estimated_simulation_parameters = simulation_parameters
    
    #delta for each parameter
    #approximately 1% or the lowest decimal point
    #[game_parameters, simulation_parameters]
    
    #mudança de aproximadamente 10% nos parâmetros
    delta = [-0.09, 0.9, 0.01, -0.09, -5.0, 3.0, 0.3]
    
    
    for i in 1:P
        if(i<=size(game_parameters,1))
            game_parameters = copy(estimated_game_parameters)
            
            game_parameters[i] = game_parameters[i] + delta[i]
            
            
        else
            simulation_parameters = copy(estimated_simulation_parameters)
            simulation_parameters[i-size(game_parameters,1)] = simulation_parameters[i-size(game_parameters,1)] + delta[i]
            
            
        end
        
        
        A2 = @spawnat aux solve_tree(data2, game_parameters)
        A1 = solve_tree(data1, game_parameters)
        A2 = fetch(A2)
        
        new_simulated_moments = simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
        
        D[:, i] = (new_simulated_moments - simulated_moments) ./ simulated_moments
        
    end
    
    
    
    return D
    
end


derivative_moments (generic function with 1 method)

In [44]:
Derivative = derivative_moments(S, N, game_parameters, simulation_parameters, simulated_moments, data1, data2, n1, aux)

       From worker 3:	 78.133493 seconds (257.31 M allocations: 5.714 GiB, 1.27% gc time)
88.565649 seconds (286.24 M allocations: 6.354 GiB, 1.35% gc time)
parameter 1[0.8400000000000001, 9.03, 0.1, 0.96][50.0, 28.0, 3.04]
227.087850 seconds (633.40 M allocations: 14.035 GiB      From worker 3:	214.143601 seconds (575.54 M allocations: 12.754 GiB, 15.63% gc time)
, 12.11% gc time)
parameter 2[0.93, 9.93, 0.1, 0.96][50.0, 28.0, 3.04]
201.762035 seconds (633.40 M allocations: 14.035 GiB      From worker 3:	185.911825 seconds (575.54 M allocations: 12.754 GiB, 1.24% gc time)
, 1.28% gc time)
parameter 3[0.93, 9.03, 0.11, 0.96][50.0, 28.0, 3.04]
227.085587 seconds (633.40 M allocations: 14.035 GiB      From worker 3:	208.538957 seconds (575.54 M allocations: 12.754 GiB, 8.34% gc time)
, 9.74% gc time)
parameter 4[0.93, 9.03, 0.1, 0.87][50.0, 28.0, 3.04]
206.798228 seconds (633.40 M allocations: 14.035 GiB      From worker 3:	190.852944 seconds (575.54 M allocations: 12.754 GiB, 1.20% gc t

8×7 Array{Float64,2}:
 -0.300285    -0.0373065    0.0327043  …   0.0254048    0.124509
 -0.00226709   0.0          0.0            0.0          0.0
 -0.31029      0.00390974   0.0115085     -0.0437529    0.00409063
 -0.640983    -0.220792    -0.380913       0.00518135  -0.00902557
 -0.0584648   -0.30347     -0.460987       1.1649       1.24341
 -0.33168     -0.163394    -0.135739   …   0.194657     0.163276
 -0.280611    -0.0351513   -0.0100293     -0.162802    -0.181693
  0.54501      0.118657     0.245964       0.0631597    0.0484278

In [45]:
Derivative[2,:]

7-element Array{Float64,1}:
 -0.002267093128077158
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [47]:
using DelimitedFiles

writedlm( "Derivative-nocd50.csv",  Derivative, ',')